# TF- IDF Vectorizer 

<font face='georgia'>
    
   <h4><strong>What does tf-idf mean?</strong></h4>

   <p>    
Tf-idf stands for <em>term frequency-inverse document frequency</em>, and the tf-idf weight is a weight often used in information retrieval and text mining. This weight is a statistical measure used to evaluate how important a word is to a document in a collection or corpus. The importance increases proportionally to the number of times a word appears in the document but is offset by the frequency of the word in the corpus. Variations of the tf-idf weighting scheme are often used by search engines as a central tool in scoring and ranking a document's relevance given a user query.
</p>
    
   <p>
One of the simplest ranking functions is computed by summing the tf-idf for each query term; many more sophisticated ranking functions are variants of this simple model.
</p>
    
   <p>
Tf-idf can be successfully used for stop-words filtering in various subject fields including text summarization and classification.
</p>
    
</font>

<font face='georgia'>
    <h4><strong>How to Compute:</strong></h4>

Typically, the tf-idf weight is composed by two terms: the first computes the normalized Term Frequency (TF), aka. the number of times a word appears in a document, divided by the total number of words in that document; the second term is the Inverse Document Frequency (IDF), computed as the logarithm of the number of the documents in the corpus divided by the number of documents where the specific term appears.

 <ul>
    <li>
<strong>TF:</strong> Term Frequency, which measures how frequently a term occurs in a document. Since every document is different in length, it is possible that a term would appear much more times in long documents than shorter ones. Thus, the term frequency is often divided by the document length (aka. the total number of terms in the document) as a way of normalization: <br>

$TF(t) = \frac{\text{Number of times term t appears in a document}}{\text{Total number of terms in the document}}.$
</li>
<li>
<strong>IDF:</strong> Inverse Document Frequency, which measures how important a term is. While computing TF, all terms are considered equally important. However it is known that certain terms, such as "is", "of", and "that", may appear a lot of times but have little importance. Thus we need to weigh down the frequent terms while scale up the rare ones, by computing the following: <br>

$IDF(t) = \log_{e}\frac{\text{Total  number of documents}} {\text{Number of documents with term t in it}}.$
for numerical stabiltiy we will be changing this formula little bit
$IDF(t) = \log_{e}\frac{\text{Total  number of documents}} {\text{Number of documents with term t in it}+1}.$
</li>
</ul>

<br>
<h4><strong>Example</strong></h4>
<p>

Consider a document containing 100 words wherein the word cat appears 3 times. The term frequency (i.e., tf) for cat is then (3 / 100) = 0.03. Now, assume we have 10 million documents and the word cat appears in one thousand of these. Then, the inverse document frequency (i.e., idf) is calculated as log(10,000,000 / 1,000) = 4. Thus, the Tf-idf weight is the product of these quantities: 0.03 * 4 = 0.12.
</p>
</font>

## Task-1

<font face='georgia'>
    <h4><strong>1. Built a TFIDF Vectorizer & compared its results with Sklearn:</strong></h4>
    <li> Sklearn does few more tweaks in the implementation of its version of TFIDF vectorizer, so to replicate the exact results added following things to the custom implementation of tfidf vectorizer:
       <ol>
        <li> Sklearn has its vocabulary generated from idf sroted in alphabetical order</li>
        <li> Sklearn formula of idf is different from the standard textbook formula. Here the constant <strong>"1"</strong> is added to the numerator and denominator of the idf as if an extra document was seen containing every term in the collection exactly once, which prevents zero divisions.
            
 $IDF(t) = 1+\log_{e}\frac{1\text{ }+\text{ Total  number of documents in collection}} {1+\text{Number of documents with term t in it}}.$
        </li>
        <li> Sklearn applies L2-normalization on its output matrix.</li>
        <li> The final output of sklearn tfidf vectorizer is a sparse matrix.</li>
    </ol>
   

### Corpus

In [21]:
## SkLearn# Collection of string documents

corpus = [
     'this is the first document',
     'this document is the second document',
     'and this is the third one',
     'is this the first document',
]

### SkLearn Implementation

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)
skl_output = vectorizer.transform(corpus)

In [23]:
# sklearn feature names, they are sorted in alphabetic order by default.

print(vectorizer.get_feature_names())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [24]:
# Here we will print the sklearn tfidf vectorizer idf values after applying the fit method
# After using the fit function on the corpus the vocab has 9 words in it, and each has its idf value.

print(vectorizer.idf_)


[1.91629073 1.22314355 1.51082562 1.         1.91629073 1.91629073
 1.         1.91629073 1.        ]


In [25]:
# shape of sklearn tfidf vectorizer output after applying transform method.

skl_output.shape

(4, 9)

In [26]:
# sklearn tfidf values for first line of the above corpus.
# Here the output is a sparse matrix

print(skl_output[0])

  (0, 8)	0.38408524091481483
  (0, 6)	0.38408524091481483
  (0, 3)	0.38408524091481483
  (0, 2)	0.5802858236844359
  (0, 1)	0.46979138557992045


In [27]:
# sklearn tfidf values for first line of the above corpus.
# To understand the output better, here we are converting the sparse output matrix to dense matrix and printing it.
# This output is normalized using L2 normalization. sklearn does this by default.

print(skl_output[0].toarray())



[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


### The custom implementation

In [28]:
from collections import Counter
from tqdm import tqdm
from scipy.sparse import csr_matrix
import math
import operator
from sklearn.preprocessing import normalize
import numpy

In [29]:
def fit(corpus):
    """
        This function finds all unique words in the data , assigns a dimension-number to each unique word 
        and returns IDF values for all those words 
    """
    
    unique_words=set()
    IDF=dict()
    no_of_docs=len(corpus)
    
    #loop to find the unique words in the corpus 
    for i in corpus:
        for j in i.split(" "):
            unique_words.add(j)
                
    unique_words=sorted(list(unique_words))
    #Assigning a dimension no to each word in corpus 
    features={i:j for j,i in enumerate(unique_words)}
    
    
    #Loop to calcuate the IDF of terms
    for i in features.keys():
        word_freq_in_docs=0
        for j in corpus:
            if i in j:
                word_freq_in_docs+=1
        IDF[i]=1+math.log((1+no_of_docs)/(1+word_freq_in_docs))
    
    
    return features,IDF




In [30]:
features,IDF=fit(corpus)
print("Features and dimension no")
print(features)
print("*"*100)
print("IDF Values")
print(IDF)

Features and dimension no
{'and': 0, 'document': 1, 'first': 2, 'is': 3, 'one': 4, 'second': 5, 'the': 6, 'third': 7, 'this': 8}
****************************************************************************************************
IDF Values
{'and': 1.916290731874155, 'document': 1.2231435513142097, 'first': 1.5108256237659907, 'is': 1.0, 'one': 1.916290731874155, 'second': 1.916290731874155, 'the': 1.0, 'third': 1.916290731874155, 'this': 1.0}


In [31]:
def transform(corpus,features,IDF):
    
    """
        This function returns a TF-IDF vector for all the documents in the dataset as a sparse matrix """

    rows=[]
    columns=[]
    values=[]
    
    #Loop to find the TF-IDF values for each document in the dataset
    for i,j in list(enumerate(corpus)):
        words_in_doc=j.split(" ")
        doc_size=len(words_in_doc)
        
        for j in set(words_in_doc):
            if features.get(j)!=None:
                #Finding the TF value
                TF=words_in_doc.count(j)/doc_size
                idf=IDF[j]
                rows.append(i)
                columns.append(features[j])
                values.append(TF*idf)
        
        #Returning a sparse matrix for the TF-IDF vectors
    return csr_matrix((values, (rows,columns)), shape=(len(corpus),len(features)))
            


In [32]:
vector=normalize(transform(corpus,features,IDF), norm='l2', axis=1, copy=True, 
                 return_norm=False)
print("Sparse Matrix")
print("*"*100)
print(vector)      
print("\nDense Matrix")
print("*"*100)
#Returning a dense matrix for the TF-IDF vectors
print(vector.toarray())

Sparse Matrix
****************************************************************************************************
  (0, 1)	0.4697913855799205
  (0, 2)	0.580285823684436
  (0, 3)	0.3840852409148149
  (0, 6)	0.3840852409148149
  (0, 8)	0.3840852409148149
  (1, 1)	0.6876235979836937
  (1, 3)	0.2810886740337529
  (1, 5)	0.5386476208856762
  (1, 6)	0.2810886740337529
  (1, 8)	0.2810886740337529
  (2, 0)	0.511848512707169
  (2, 3)	0.267103787642168
  (2, 4)	0.511848512707169
  (2, 6)	0.267103787642168
  (2, 7)	0.511848512707169
  (2, 8)	0.267103787642168
  (3, 1)	0.4697913855799205
  (3, 2)	0.580285823684436
  (3, 3)	0.3840852409148149
  (3, 6)	0.3840852409148149
  (3, 8)	0.3840852409148149

Dense Matrix
****************************************************************************************************
[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]
 [0.         0.6876236  0.         0.28108867 0.         0.53864762
  0.28108867 0.         0.

In [33]:
# Below is the code to load the cleaned_strings pickle file provided
# Here corpus is of list type

import pickle
with open('cleaned_strings', 'rb') as f:
    corpus = pickle.load(f)
    
# printing the length of the corpus loaded
print("Number of documents in corpus = ",len(corpus))

Number of documents in corpus =  746


In [34]:

def fit(corpus):
    """
        This function finds all unique words in the data , assigns a dimension-number to each unique word 
        and returns top 50 IDF values for corresponding words 
    """
    
    unique_words=set()
    IDF_list=[]
    no_of_docs=len(corpus)
    
    #loop to find the unique words
    for i in corpus:
        for j in i.split(" "):
            unique_words.add(j)
                
    unique_words=sorted(list(unique_words))
    
    
    #Loop to calcuate the IDF of terms
    for i in unique_words:
        word_freq_in_docs=0
        for j in corpus:
            if i in j:
                word_freq_in_docs+=1
        IDF_list.append(1+math.log((1+no_of_docs)/(1+word_freq_in_docs)))
    
    #Zipping words and IDF values 
    IDF_zipped=list(zip(unique_words,IDF_list))
    #Sorting in descending order by IDF values 
    res = sorted(IDF_zipped, key = operator.itemgetter(1),reverse=True)
    #Taking the top 50 IDF values 
    res=res[:50]
    #Unzipping words and IDF values
    unique_words,IDF_values=list(zip(*res))
    #Creating a features vector
    features={i:j for j,i in enumerate(unique_words)}
    #Creating a IDF vector 
    IDF={i:j for i,j in res}
   
    return features,IDF






In [35]:
features,IDF=fit(corpus)
print("Features and dimension no")
print(features)
print("*"*100)
print("IDF Values")
print(IDF)

Features and dimension no
{'aailiyah': 0, 'abandoned': 1, 'abroad': 2, 'abstruse': 3, 'academy': 4, 'accents': 5, 'accessible': 6, 'acclaimed': 7, 'accolades': 8, 'accurately': 9, 'achille': 10, 'ackerman': 11, 'adams': 12, 'added': 13, 'admins': 14, 'admiration': 15, 'admitted': 16, 'adrift': 17, 'adventure': 18, 'aesthetically': 19, 'affected': 20, 'affleck': 21, 'afternoon': 22, 'agreed': 23, 'aimless': 24, 'aired': 25, 'akasha': 26, 'alert': 27, 'alike': 28, 'allison': 29, 'allowing': 30, 'alongside': 31, 'amateurish': 32, 'amazed': 33, 'amazingly': 34, 'amusing': 35, 'amust': 36, 'anatomist': 37, 'angela': 38, 'angelina': 39, 'angry': 40, 'anguish': 41, 'angus': 42, 'animals': 43, 'animated': 44, 'anita': 45, 'anniversary': 46, 'anthony': 47, 'antithesis': 48, 'anyway': 49}
****************************************************************************************************
IDF Values
{'aailiyah': 6.922918004572872, 'abandoned': 6.922918004572872, 'abroad': 6.922918004572872, 'abst

In [36]:
def transform(corpus,features,IDF):
    
    """
        This function returns a TF-IDF vector for all the documents in the dataset as a sparse matrix with 50 features"""
   
    rows=[]
    columns=[]
    values=[]
    
    #Loop to find the TF-IDF values for each document in the dataset
    for i,j in list(enumerate(corpus)):
        words_in_doc=j.split(" ")
        doc_size=len(words_in_doc)
        for j in set(words_in_doc):
            #If words are not there in the top 50 features ignore those words 
              if features.get(j)!=None:
                #Finding the TF value
                TF=words_in_doc.count(j)/doc_size
                idf=IDF[j]
                rows.append(i)
                columns.append(features[j])
                values.append(TF*idf)
        #Returning a sparse matrix for the TF-IDF vectors
    return csr_matrix((values, (rows,columns)), shape=(len(corpus),len(features)))
        

In [37]:
vector=normalize(transform(corpus,features,IDF), norm='l2', axis=1, copy=True, 
                 return_norm=False)
print("Sparse Matrix")
print("*"*100)
print(vector)      
print("\nDense Matrix")
print("*"*100)
#Dense matrix 
print(vector.toarray())

Sparse Matrix
****************************************************************************************************
  (0, 24)	1.0
  (19, 43)	1.0
  (68, 21)	1.0
  (72, 23)	1.0
  (74, 25)	1.0
  (89, 47)	1.0
  (135, 3)	0.37796447300922725
  (135, 9)	0.37796447300922725
  (135, 15)	0.37796447300922725
  (135, 17)	0.37796447300922725
  (135, 29)	0.37796447300922725
  (135, 32)	0.37796447300922725
  (135, 40)	0.37796447300922725
  (176, 39)	1.0
  (192, 18)	1.0
  (193, 20)	1.0
  (216, 2)	1.0
  (225, 16)	1.0
  (227, 14)	1.0
  (241, 35)	1.0
  (270, 1)	1.0
  (290, 22)	1.0
  (341, 34)	1.0
  (344, 33)	1.0
  (348, 8)	1.0
  (409, 5)	1.0
  (430, 31)	1.0
  (457, 36)	1.0
  (461, 4)	0.7071067811865475
  (461, 44)	0.7071067811865475
  (465, 30)	1.0
  (475, 28)	1.0
  (493, 6)	1.0
  (500, 38)	1.0
  (544, 41)	1.0
  (548, 0)	0.7071067811865475
  (548, 26)	0.7071067811865475
  (608, 12)	1.0
  (612, 10)	1.0
  (620, 37)	0.7071067811865476
  (620, 42)	0.7071067811865476
  (632, 7)	1.0
  (644, 11)	0.57735026918962